In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import youtokentome as yttm
import tensorflow as tf

2022-09-17 18:21:55.980159: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
import json

with open('train-syllable.json') as fopen:
    data = json.load(fopen)
    
train_X = [d[0] for d in data]
train_Y = [d[1] for d in data]

In [4]:
with open('test-syllable.json') as fopen:
    data = json.load(fopen)
    
test_X = [d[0] for d in data]
test_Y = [d[1] for d in data]

In [5]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(train_X + train_Y + test_X + test_Y))

In [6]:
_ = yttm.BPE.train(data='text.txt', vocab_size=300, model='syllable.model',
              pad_id=0, unk_id=2, bos_id=3, eos_id=1)

Training parameters
  input: text.txt
  model: syllable.model
  vocab_size: 300
  n_threads: 8
  character_coverage: 1
  pad: 0
  unk: 2
  bos: 3
  eos: 1

reading file...
learning bpe...
number of unique characters in the training data: 28
number of deleted characters: 0
number of unique characters left: 28
model saved to: syllable.model


In [7]:
bpe = yttm.BPE(model='syllable.model')

In [8]:
bpe.vocab_size()

300

In [9]:
PAD = 0
EOS = 1
UNK = 2
GO = 3

In [10]:
class Translator:
    def __init__(self, size_layer, num_layers, embedded_size,
                 from_dict_size, to_dict_size, learning_rate, beam_width = 10):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        def attention(encoder_out, seq_len, reuse=False):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = encoder_out,
                                                                    memory_sequence_length = seq_len)
            return tf.contrib.seq2seq.AttentionWrapper(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells(reuse) for _ in range(num_layers)]), 
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        encoder_embedding = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embedding = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        
        encoder_out, encoder_state = tf.nn.dynamic_rnn(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
            inputs = tf.nn.embedding_lookup(encoder_embedding, self.X),
            sequence_length = self.X_seq_len,
            dtype = tf.float32)
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        dense = tf.layers.Dense(to_dict_size)
        
        with tf.variable_scope('decode'):
            decoder_cells = attention(encoder_out, self.X_seq_len)
            training_helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = tf.nn.embedding_lookup(decoder_embedding, decoder_input),
                sequence_length = self.Y_seq_len,
                time_major = False)
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell = decoder_cells,
                    helper = training_helper,
                    initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                    output_layer = dense)
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder = training_decoder,
                    impute_finished = True,
                    maximum_iterations = tf.reduce_max(self.Y_seq_len))
            self.training_logits = training_decoder_output.rnn_output
            
        with tf.variable_scope('decode', reuse=True):
            predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding = decoder_embedding,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS)
            predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell = decoder_cells,
                    helper = predicting_helper,
                    initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                    output_layer = dense)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder = predicting_decoder,
                    impute_finished = True,
                    maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            self.greedy = predicting_decoder_output.sample_id
            self.greedy = tf.identity(self.greedy,name='greedy')
        
        with tf.variable_scope('decode', reuse=True):
            
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_out, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            decoder_cell = attention(encoder_out_tiled, X_seq_len_tiled, reuse=True)
            
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = decoder_cell,
                embedding = decoder_embedding,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = decoder_cell.zero_state(batch_size * beam_width, tf.float32).clone(
                    cell_state = encoder_state_tiled),
                beam_width = beam_width,
                output_layer = dense,
                length_penalty_weight = 0.0)
            
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = tf.reduce_max(self.X_seq_len))
            
            self.beam = predicting_decoder_output.predicted_ids[:, :, 0]
            self.beam = tf.identity(self.beam,name='beam')
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
size_layer = 256
num_layers = 2
embedded_size = 256
learning_rate = 5e-4
batch_size = 32
epoch = 20

In [12]:
vocab_size = bpe.vocab_size()

In [13]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, vocab_size, vocab_size, learning_rate)
sess.run(tf.global_variables_initializer())







The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2022-09-17 18:21:58.373496: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2112000000 Hz
2022-09-17 18:21:58.373933: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3535f30 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-09-17 18:21:58.373948: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-09-17 18:21:58.375027: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-17 18:21:58.377049: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-17 18:21:58.377070: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-09-17 18:21:58.377074: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2

In [14]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'lstm/model.ckpt')

'lstm/model.ckpt'

In [15]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [16]:
train_X[:10], train_Y[:10]

(['adpis',
  'terbongkang',
  'memijit',
  'ordi',
  'dinaskan',
  'plag',
  'pendayung',
  'sambung-bersambung',
  'prausul',
  'betari'],
 ['ad.pis',
  'ter.bong.kang',
  'me.mi.jit',
  'or.di',
  'di.nas.kan',
  'plag',
  'pen.da.yung',
  'sam.bung.ber.sam.bung',
  'pra.u.sul',
  'be.ta.ri'])

In [17]:
def encode(strings):
    encoded = bpe.encode(strings, output_type=yttm.OutputType.ID)
    for i in range(len(encoded)):
        encoded[i].append(1)
        
    return pad_sentence_batch(encoded, 0)[0]

In [18]:
batch_x = encode(train_X[: 5])
batch_y = encode(train_Y[: 5])

In [19]:
sess.run([model.cost, model.optimizer], feed_dict = {model.X: batch_x, model.Y: batch_y})

[5.6956706, None]

In [20]:
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: encode(test_X[:1])})

In [52]:
import tqdm
import numpy as np

batch_size = 8

for e in range(1):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = encode(train_X[i: index])
        batch_y = encode(train_Y[i: index])
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
        
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = encode(test_X[i: index])
        batch_y = encode(test_Y[i: index])
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop: 100%|█| 4635/4635 [01:45<00:00, 43.81it/s, accuracy=1, cost=0.00
minibatch loop: 100%|█| 244/244 [00:01<00:00, 230.19it/s, accuracy=1, cost=0.001

epoch 1, training avg loss 0.018719, training avg acc 0.995121
epoch 1, testing avg loss 0.079549, testing avg acc 0.988486


In [30]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'lstm/model.ckpt')

'lstm/model.ckpt'

In [54]:
berjalankan = bpe.encode('memijit', output_type=yttm.OutputType.ID) + [1]
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: [berjalankan]})
bpe.decode(greedy.tolist())

['me.mi.jit<EOS>']

In [55]:
berjalankan = bpe.encode('husein', output_type=yttm.OutputType.ID) + [1]
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: [berjalankan]})
bpe.decode(greedy.tolist())

['hu.sein<EOS>']

In [56]:
berjalankan = bpe.encode('ayam-ayam', output_type=yttm.OutputType.ID) + [1]
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: [berjalankan]})
bpe.decode(greedy.tolist())

['a.yam.a.yam<EOS>']

In [57]:
berjalankan = bpe.encode('kesedapan', output_type=yttm.OutputType.ID) + [1]
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: [berjalankan]})
bpe.decode(greedy.tolist())

['ke.se.da.pan<EOS>']

In [48]:
def replace_same_length(l, r):
    l = l.replace('-', '')
    if len(l) != len(r.replace('.', '')):
        return False, r
    
    index = {}
    i = 0
    for no, c in enumerate(r):
        if c not in '.O':
            index[no] = i
            i += 1
    index = {v: k for k, v in index.items()}
    
    r = list(r)
    
    for no, c in enumerate(l):
        if c != r[index[no]]:
            r[index[no]] = c
            
    return True, ''.join(r)

replace_same_length('biadat', 'bi.Oa.dat'), replace_same_length('sompek', 'som.pe/')

((False, 'bi.Oa.dat'), (True, 'som.pek'))

In [49]:
replace_same_length('ayam-ayam', 'a.yau.a.yam')

(True, 'a.yam.a.yam')

In [50]:
def calculate_cer(actual, hyp):
    """
    Calculate CER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    actual = actual.replace(' ', '')
    hyp = hyp.replace(' ', '')
    return Lev.distance(actual, hyp) / len(actual)


def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [59]:
cers, wers = [], []

batch_size = 1
pbar = tqdm.tqdm(
    range(0, len(test_X), batch_size), desc = 'minibatch loop')
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x = encode(test_X[i: index])
    batch_y = encode(test_Y[i: index])
    feed = {model.X: batch_x,
            model.Y: batch_y,}
    greedy = sess.run(model.greedy, feed_dict = feed)
    predicted = bpe.decode(greedy.tolist())[0].replace('<EOS>', '')
    actual = bpe.decode(batch_y)[0].replace('<EOS>', '')
    cer = calculate_cer(actual, predicted)
    wer = calculate_wer(actual, predicted)
    
    cers.append(cer)
    wers.append(wer)
    
np.mean(cers), np.mean(wers)

minibatch loop: 100%|██████████████████████| 1952/1952 [00:03<00:00, 595.79it/s]


(0.007769342911655472, 0.0430327868852459)

In [ ]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'lstm-8/model.ckpt')

In [58]:
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'lstm-8/model.ckpt')

INFO:tensorflow:Restoring parameters from lstm-8/model.ckpt


In [60]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'OptimizeLoss' not in n.name
        and 'Global_Step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Variable',
 'Variable_1',
 'rnn/multi_rnn_cell/cell_0/lstm_cell/kernel',
 'rnn/multi_rnn_cell/cell_0/lstm_cell/bias',
 'rnn/multi_rnn_cell/cell_1/lstm_cell/kernel',
 'rnn/multi_rnn_cell/cell_1/lstm_cell/bias',
 'decode/memory_layer/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/bias',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_1/lstm_cell/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_1/lstm_cell/bias',
 'decode/decoder/attention_wrapper/bahdanau_attention/query_layer/kernel',
 'decode/decoder/attention_wrapper/bahdanau_attention/attention_v',
 'decode/decoder/attention_wrapper/attention_layer/kernel',
 'decode/decoder/dense/kernel',
 'decode/decoder/dense/bias',
 'decode_1/greedy',
 'decode_2/decoder/while/BeamSearchDecoderStep/beam_width',
 'decode_2/decoder/while/BeamSearchDecoderStep/next_beam_probs/range/start',
 

In [61]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"
    clear_devices = True
    with tf.Session(graph=tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(",")
        ) 
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

In [62]:
freeze_graph("lstm-8", strings)




INFO:tensorflow:Restoring parameters from lstm-8/model.ckpt
INFO:tensorflow:Froze 16 variables.
INFO:tensorflow:Converted 16 variables to const ops.

1649 ops in the final graph.


In [63]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [64]:
g = load_graph('lstm-8/frozen_model.pb')

In [65]:
x = g.get_tensor_by_name('import/Placeholder:0')
i_greedy = g.get_tensor_by_name('import/decode_1/greedy:0')
i_beam = g.get_tensor_by_name('import/decode_2/beam:0')

In [66]:
test_sess = tf.InteractiveSession(graph=g)

/home/husein/tf-nvidia/lib/python3.8/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [67]:
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.contrib.seq2seq.python.ops import beam_search_ops

In [68]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'lstm-8/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                       ['Placeholder'],
                                       ['decode_1/greedy', 'decode_2/beam'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

2022-09-17 18:43:20.522645: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying add_default_attributes
2022-09-17 18:43:20.529094: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2022-09-17 18:43:20.534943: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_1/greedy
2022-09-17 18:43:20.535663: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_2/beam
2022-09-17 18:43:20.543275: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_1/greedy
2022-09-17 18:43:20.543789: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_2/beam
2022-09-17 18:43:20.549963: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_1/greedy
2022-09-17 18:43:20.550526: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_2/beam
2022-09-17 18:43:20.569831

In [69]:
from malaya_boilerplate.huggingface import upload_dict

/home/husein/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [70]:
!tar -cvf output-syllable-base.tar lstm-8

lstm-8/
lstm-8/frozen_model.pb.quantized
lstm-8/frozen_model.pb
lstm-8/model.ckpt.meta
lstm-8/checkpoint
lstm-8/model.ckpt.index
lstm-8/model.ckpt.data-00000-of-00001


In [71]:
files_mapping = {'output-syllable-base.tar': 'output-syllable-base.tar'}
upload_dict(model = 'pretrained-syllable', files_mapping = files_mapping)

/home/husein/tf-nvidia/lib/python3.8/site-packages/huggingface_hub/hf_api.py:91: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.10. Pass `repo_id` instead.
  warnings.warn(


In [72]:
files_mapping = {'lstm-8/frozen_model.pb': 'model.pb'}
upload_dict(model = 'syllable-lstm-bahdanau-base', files_mapping = files_mapping)

In [73]:
files_mapping = {'lstm-8/frozen_model.pb.quantized': 'model.pb'}
upload_dict(model = 'syllable-lstm-bahdanau-base-quantized', files_mapping = files_mapping)